In [ ]:
# My OpenAI Key
import os
os.environ['OPENAI_API_KEY'] = "INSERT OPENAI KEY"

In [1]:
from gpt_index.index import GPTIndex

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [ ]:
index = GPTIndex.from_input_dir('data')

In [ ]:
index.save_to_disk('index.json')

In [2]:
# try loading
new_index = GPTIndex.load_from_disk('index.json')

In [4]:
# try verbose=True for more detailed outputs
new_index.query("What did the author do growing up?")

> Starting query: What did the author do growing up?
> Selected node: 
ANSWER: 1

This summary was selected because it provides the most information about the author's background and what they did growing up. It discusses their experience with writing and programming before college, their studies in college, and their switch to Lisp programming. It also mentions their interest in art and their application to art school.
> Node Summary text:  The individual worked on writing and programming before college. They attempted to write essays and stories, but they were not very good. In college, they studied philosophy but found it unfulfilling. They then switched to AI but realized that it was not going to work. They decided to focus on Lisp instead and wrote a book about it. They then took up art and applied to art school. They were accepted to RISD and then later to the Accademia di Belli Arti in Florence. They took the entrance exam and was accepted. They then had to learn Italian.
> Sele

'The author grew up writing short stories and programming on an IBM 1401.'

In [3]:
# try verbose=True for more detailed outputs
new_index.query("What did the author do after his time at Y Combinator?")

> Starting query: What did the author do after his time at Y Combinator?
> Selected node: 
ANSWER: 3

This summary was selected because it provides the most information about the author's post-Y Combinator activities. It describes how the author started a new company and wrote a book on Lisp programming.
> Node Summary text:  The individual describes their experience working on a new website generator in Lisp with a partner. They explain how they came up with the idea to run the software on a server and allow users to control it through clicking links in a browser. They recount how they started a new company, Viaweb, with the goal of being easy and inexpensive to use. They describe how they recruited additional programmers and opened for business in January 1996. They explain how they realized that growth rate is the most important metric for a startup and recount how they reached breakeven a few years later. They describe how they sold the company to Yahoo in 1998 and how they used th

'The author left Yahoo to start a new startup.'